In [55]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install contractions
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 16.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=3f0af2986478379b991e21325b14fb7ada16f10a54ba25c9469387637480f380
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl siz

In [4]:
import pandas as pd
import unicodedata
import re
import contractions
import string

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim_models

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [8]:
df = pd.read_parquet('/content/drive/MyDrive/final_twitter_data.parquet')
df.head()

,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type
0,2021-06-20T21:33:30.000Z,1406726930888540160,1406726930888540160,2981738470,RT @davidlohner: #IchbinHanna und schreibe auc...,7,0,0,0,"[{'type': 'retweeted', 'id': '1406714909258362...",['IchbinHanna'],retweet
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,Wenn Wirtschaftsjounalist:innen über #IchbinHa...,9,1,62,4,[],['IchbinHanna'],original
2,2021-06-20T20:50:42.000Z,1406716159064485895,1406716159064485895,2981738470,RT @DrKEichhorn: Wieder ein Artikel ohne echte...,16,0,0,0,"[{'type': 'retweeted', 'id': '1406697315159752...",[],retweet
3,2021-06-20T17:56:03.000Z,1406672208056012807,1406672208056012807,2981738470,RT @chrismeyer2203: So sieht es aus ⬇️ oder an...,7,0,0,0,"[{'type': 'retweeted', 'id': '1406175728190865...",[],retweet
4,2021-06-20T17:53:00.000Z,1406671439328055301,1406671439328055301,2981738470,RT @steffen_siegel: Modelle einer Universitäts...,17,0,0,0,"[{'type': 'retweeted', 'id': '1406666454368493...",[],retweet


In [9]:
new_df = df.loc[df['tweet_type'] != 'retweet']
new_df.head()

,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,Wenn Wirtschaftsjounalist:innen über #IchbinHa...,9,1,62,4,[],['IchbinHanna'],original
5,2021-06-20T16:17:25.000Z,1406647386542325764,1406647386542325764,2981738470,☝️⬇️ #IchbinHanna #PeerReview https://t.co/PFN...,0,0,1,0,"[{'type': 'quoted', 'id': '1406620276822061057'}]","['IchbinHanna', 'PeerReview']",original
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,Thread ⬇️ about the cruel #precarity in German...,5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,"Liebe @unidue und Forsa - schön, dass ihr euch...",0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,Wichtiger Thread ⬇️ zu #Machtmissbrauch in der...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original


In [10]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

new_df.text = new_df.text.apply(remove_emojis)
new_df.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,Wenn Wirtschaftsjounalist:innen über #IchbinHa...,9,1,62,4,[],['IchbinHanna'],original
5,2021-06-20T16:17:25.000Z,1406647386542325764,1406647386542325764,2981738470,#IchbinHanna #PeerReview https://t.co/PFNrovEOAS,0,0,1,0,"[{'type': 'quoted', 'id': '1406620276822061057'}]","['IchbinHanna', 'PeerReview']",original
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,Thread about the cruel #precarity in German #...,5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,"Liebe @unidue und Forsa - schön, dass ihr euch...",0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,Wichtiger Thread zu #Machtmissbrauch in der #...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original


In [14]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=02292ab57a716112f68cadb49fdbe23cf99c1008a6c4762bc2042a75b42b200e
  Stored in directory: /root/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


In [15]:
from langdetect import detect

In [16]:
def detect_my(text):
  try:
    return detect(text)
  except:
    return "unknown"

In [17]:
new_df['language']=new_df['text'].apply(detect_my)

<ipython-input-17-b811955c4c3d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['language']=new_df['text'].apply(detect_my)


In [24]:
Df = new_df.drop_duplicates(subset=['tweet_id'])

In [25]:
Df['language'].value_counts()

de         42734
en          6137
sv          1161
tl           203
unknown      122
fr           109
it            81
id            59
nl            55
es            38
af            23
fi            18
pl            18
cy            15
no            15
da            15
so            14
sw            13
et            13
ca            12
pt            10
tr             8
cs             7
ar             5
sk             4
hi             2
vi             2
fa             2
hr             2
hu             2
ro             2
th             1
sq             1
sl             1
ru             1
lv             1
Name: language, dtype: int64

In [27]:
Df.shape

(50906, 13)

In [28]:
Df

,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type,language
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,Wenn Wirtschaftsjounalist:innen über #IchbinHa...,9,1,62,4,[],['IchbinHanna'],original,de
5,2021-06-20T16:17:25.000Z,1406647386542325764,1406647386542325764,2981738470,#IchbinHanna #PeerReview https://t.co/PFNrovEOAS,0,0,1,0,"[{'type': 'quoted', 'id': '1406620276822061057'}]","['IchbinHanna', 'PeerReview']",original,nl
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,Thread about the cruel #precarity in German #...,5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original,en
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,"Liebe @unidue und Forsa - schön, dass ihr euch...",0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original,de
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,Wichtiger Thread zu #Machtmissbrauch in der #...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...
308293,2021-09-17T14:36:31.000Z,1438874511605125123,1438766234489794560,743696568939753475,@asayeed @mjpost I still can't get over how th...,0,1,0,0,"[{'type': 'replied_to', 'id': '143887433770928...",[],original,en
308311,2021-08-25T04:56:31.000Z,1430393629651177473,1430093188757401618,1134768684315160577,@VinciGusta WissZeitVG Maximaldauer erreicht?\...,0,0,0,0,"[{'type': 'replied_to', 'id': '143009318875740...",[],original,de
308323,2022-08-13T13:39:47.000Z,1558448232631762944,1558448232631762944,329984940,fellow profs: #academicprecarity is OUR proble...,2,0,4,0,"[{'type': 'quoted', 'id': '1556700609155317761'}]",['academicprecarity'],original,en
308326,2021-09-02T19:41:11.000Z,1433515364932067338,1433515364932067338,3005636663,"@AGH_Berlin beschließt neues ""Gesetz zur Stärk...",1,0,6,0,[],"['Wissenschaft', 'Postdocs', 'Promotionsrecht'...",original,de


In [31]:
de_df = Df[Df.language == 'de']
de_df.shape

(42734, 13)

In [33]:
de_df

,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type,language
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,Wenn Wirtschaftsjounalist:innen über #IchbinHa...,9,1,62,4,[],['IchbinHanna'],original,de
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,"Liebe @unidue und Forsa - schön, dass ihr euch...",0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original,de
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,Wichtiger Thread zu #Machtmissbrauch in der #...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original,de
21,2021-06-18T07:52:36.000Z,1405795569273131011,1405795569273131011,1132055796571877376,Wir können ja mal einen #FollowFriday für #Ich...,4,0,16,0,"[{'type': 'quoted', 'id': '1405617868734148610'}]","['FollowFriday', 'IchbinHanna', 'WissZeitVG', ...",original,de
22,2021-06-18T02:42:56.000Z,1405717636999155717,1405717636999155717,2981738470,"Das ist übrigens ein Punkt, der mir bei #Ichbi...",1,0,6,0,"[{'type': 'quoted', 'id': '1405460325437149186'}]",['IchbinHanna'],original,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...
308278,2022-08-15T09:31:53.000Z,1559110620972285953,1559109649995124738,151775715,"Hätte nicht gedacht, dass ich das nochmal sage...",0,2,13,0,"[{'type': 'replied_to', 'id': '155910964999512...","['IchBinNichtMehrHanna', 'IchBinHanna']",original,de
308279,2022-08-15T09:28:02.000Z,1559109649995124738,1559109649995124738,151775715,Frisch zurück aus dem Urlaub gibt's some perso...,0,25,108,1,[],[],original,de
308311,2021-08-25T04:56:31.000Z,1430393629651177473,1430093188757401618,1134768684315160577,@VinciGusta WissZeitVG Maximaldauer erreicht?\...,0,0,0,0,"[{'type': 'replied_to', 'id': '143009318875740...",[],original,de
308326,2021-09-02T19:41:11.000Z,1433515364932067338,1433515364932067338,3005636663,"@AGH_Berlin beschließt neues ""Gesetz zur Stärk...",1,0,6,0,[],"['Wissenschaft', 'Postdocs', 'Promotionsrecht'...",original,de


In [32]:
en_df = Df[Df.language == 'en']
en_df.shape

(6137, 13)

In [34]:
en_df

,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type,language
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,Thread about the cruel #precarity in German #...,5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original,en
105,2021-06-12T21:15:23.000Z,1403823267115376649,1403823267115376649,1132055796571877376,Thread summarizing a video by the German Fede...,1,0,9,1,"[{'type': 'quoted', 'id': '1403006909079293953'}]","['IchbinHanna', 'academia', 'Germany', 'toxica...",original,en
125,2021-06-11T22:32:52.000Z,1403480380632674307,1403480380632674307,1132055796571877376,On job #precarity in German #academia and it's...,0,0,1,0,"[{'type': 'quoted', 'id': '1402963684691955714'}]","['precarity', 'academia', 'family', 'mentalhea...",original,en
215,2021-06-10T19:07:52.000Z,1403066401611472902,1403066401611472902,1132055796571877376,#AcademicTwitter @AcademicChatter #acertaindeg...,0,0,1,0,"[{'type': 'quoted', 'id': '1403059508021284864'}]","['AcademicTwitter', 'acertaindegreeofflexibili...",original,en
238,2021-06-10T15:04:44.000Z,1403005214974640143,1403005214974640143,1132055796571877376,#IchbinHanna: German researchers without a per...,7,0,15,0,"[{'type': 'quoted', 'id': '1402994337009442820'}]","['IchbinHanna', 'academia', 'AcademicTwitter',...",original,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
308092,2022-10-04T14:01:01.000Z,1577297743663583232,1577176553666273280,2474312910,@astrodicticum #IchbinHanna,0,0,0,0,"[{'type': 'replied_to', 'id': '157717655366627...",['IchbinHanna'],original,en
308243,2022-09-13T08:48:00.000Z,1569608824117465088,1569608824117465088,1554860518900486144,The @HIFMB_OL together with the @SociedadCerfa...,3,0,9,0,[],['WissZeitGesetz'],original,en
308292,2021-09-17T14:38:18.000Z,1438874960110424072,1438766234489794560,32800037,@ryandcotterell @mjpost there's no studies on ...,0,1,0,0,"[{'type': 'replied_to', 'id': '143887451160512...",[],original,en
308293,2021-09-17T14:36:31.000Z,1438874511605125123,1438766234489794560,743696568939753475,@asayeed @mjpost I still can't get over how th...,0,1,0,0,"[{'type': 'replied_to', 'id': '143887433770928...",[],original,en


### GERMAN PRE-PROCESS

In [36]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('german'))

['aber', 'alle', 'allem', 'allen', 'aller', 'alles', 'als', 'also', 'am', 'an', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'anderes', 'anderm', 'andern', 'anderr', 'anders', 'auch', 'auf', 'aus', 'bei', 'bin', 'bis', 'bist', 'da', 'damit', 'dann', 'der', 'den', 'des', 'dem', 'die', 'das', 'dass', 'daß', 'derselbe', 'derselben', 'denselben', 'desselben', 'demselben', 'dieselbe', 'dieselben', 'dasselbe', 'dazu', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'denn', 'derer', 'dessen', 'dich', 'dir', 'du', 'dies', 'diese', 'diesem', 'diesen', 'dieser', 'dieses', 'doch', 'dort', 'durch', 'ein', 'eine', 'einem', 'einen', 'einer', 'eines', 'einig', 'einige', 'einigem', 'einigen', 'einiger', 'einiges', 'einmal', 'er', 'ihn', 'ihm', 'es', 'etwas', 'euer', 'eure', 'eurem', 'euren', 'eurer', 'eures', 'für', 'gegen', 'gewesen', 'hab', 'habe', 'haben', 'hat', 'hatte', 'hatten', 'hier', 'hin', 'hinter', 'ich', 'mich', 'mir', 'ihr', 'ihre', 'ihrem', 'ihren', 'ihrer', 'ihres', 'euc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [47]:
def remove_mentions_and_tags(text): #Removing mentions and hashtags
    text = re.sub(r'@\S*', '', text)
    return re.sub(r'#\S*', '', text)

de_df.text = de_df.text.apply(remove_mentions_and_tags)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [48]:
de_df['text'] = de_df['text'].str.replace(r'[^\w\s]+', '')

<ipython-input-48-036435de67c9>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df['text'] = de_df['text'].str.replace(r'[^\w\s]+', '')
<ipython-input-48-036435de67c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_df['text'] = de_df['text'].str.replace(r'[^\w\s]+', '')


In [50]:
def to_lowercase(text):
    return text.lower()

de_df.text = de_df.text.apply(to_lowercase)

de_df.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type,language
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,wenn wirtschaftsjounalistinnen schreiben amp d...,9,1,62,4,[],['IchbinHanna'],original,de
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,liebe forsa schön per umfrage interessiert die...,0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original,de
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,wichtiger thread arbeitsverträge begünstigt ht...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original,de
21,2021-06-18T07:52:36.000Z,1405795569273131011,1405795569273131011,1132055796571877376,wir ja mal wer macht httpstcosjdilpwjwj,4,0,16,0,"[{'type': 'quoted', 'id': '1405617868734148610'}]","['FollowFriday', 'IchbinHanna', 'WissZeitVG', ...",original,de
22,2021-06-18T02:42:56.000Z,1405717636999155717,1405717636999155717,2981738470,das übrigens punkt einfach kurz kommt die akad...,1,0,6,0,"[{'type': 'quoted', 'id': '1405460325437149186'}]",['IchbinHanna'],original,de


https://www.thoughtco.com/capitalization-in-german-4069437 

In [67]:
de_df['text'] = de_df['text'].str.replace('\d+', '')

<>:1: DeprecationWarning: invalid escape sequence \d
<>:1: DeprecationWarning: invalid escape sequence \d
<ipython-input-67-62a4851892e5>:1: DeprecationWarning: invalid escape sequence \d
  de_df['text'] = de_df['text'].str.replace('\d+', '')
<ipython-input-67-62a4851892e5>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df['text'] = de_df['text'].str.replace('\d+', '')
<ipython-input-67-62a4851892e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_df['text'] = de_df['text'].str.replace('\d+', '')


In [74]:
#from sklearn.feature_extraction.text import CountVectorizer

german_stop_words = stopwords.words('german')
en_stop_words = stopwords.words('english')

def stop_word_de_removal(x):
    token = x.split()
    return ' '.join([w for w in token if not w in german_stop_words])
def stop_word_en_removal(x):
    token = x.split()
    return ' '.join([w for w in token if not w in en_stop_words])

de_df.text = de_df.text.apply(stop_word_de_removal)
de_df.text = de_df.text.apply(stop_word_en_removal)
de_df.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type,language,tokens
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,wirtschaftsjounalistinnen schreiben amp geschi...,9,1,62,4,[],['IchbinHanna'],original,de,"[wirtschaftsjounalistinnen, schreiben, amp, ge..."
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,liebe forsa schön per umfrage interessiert fra...,0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original,de,"[liebe, forsa, schön, per, umfrage, interessie..."
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,wichtiger thread arbeitsverträge begünstigt ht...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original,de,"[wichtiger, thread, arbeitsverträge, begünstig..."
21,2021-06-18T07:52:36.000Z,1405795569273131011,1405795569273131011,1132055796571877376,ja mal wer macht httpstcosjdilpwjwj,4,0,16,0,"[{'type': 'quoted', 'id': '1405617868734148610'}]","['FollowFriday', 'IchbinHanna', 'WissZeitVG', ...",original,de,"[ja, mal, wer, macht, httpstcosjdilpwjwj]"
22,2021-06-18T02:42:56.000Z,1405717636999155717,1405717636999155717,2981738470,übrigens punkt einfach kurz kommt akademische ...,1,0,6,0,"[{'type': 'quoted', 'id': '1405460325437149186'}]",['IchbinHanna'],original,de,"[übrigens, punkt, einfach, kurz, kommt, akadem..."


### LDA on german corpus

In [75]:
def generate_tokens(tweet):
  words=[]
  for word in tweet.split(' '):
  # using the if condition because we introduced extra spaces during text cleaning
    if word!='':
      words.append(word)
  return words
#storing the generated tokens in a new column named 'words'
de_df['tokens'] = de_df.text.apply(generate_tokens)
de_df.head()

<ipython-input-75-00ff416bff71>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_df['tokens'] = de_df.text.apply(generate_tokens)


,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type,language,tokens
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,wirtschaftsjounalistinnen schreiben amp geschi...,9,1,62,4,[],['IchbinHanna'],original,de,"[wirtschaftsjounalistinnen, schreiben, amp, ge..."
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,liebe forsa schön per umfrage interessiert fra...,0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original,de,"[liebe, forsa, schön, per, umfrage, interessie..."
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,wichtiger thread arbeitsverträge begünstigt ht...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original,de,"[wichtiger, thread, arbeitsverträge, begünstig..."
21,2021-06-18T07:52:36.000Z,1405795569273131011,1405795569273131011,1132055796571877376,ja mal wer macht httpstcosjdilpwjwj,4,0,16,0,"[{'type': 'quoted', 'id': '1405617868734148610'}]","['FollowFriday', 'IchbinHanna', 'WissZeitVG', ...",original,de,"[ja, mal, wer, macht, httpstcosjdilpwjwj]"
22,2021-06-18T02:42:56.000Z,1405717636999155717,1405717636999155717,2981738470,übrigens punkt einfach kurz kommt akademische ...,1,0,6,0,"[{'type': 'quoted', 'id': '1405460325437149186'}]",['IchbinHanna'],original,de,"[übrigens, punkt, einfach, kurz, kommt, akadem..."


In [76]:
def create_dictionary(words):
  return corpora.Dictionary(words)
#passing the dataframe column having tokens as the argument
de_id2word = create_dictionary(de_df.tokens)
print(de_id2word)

Dictionary(82071 unique tokens: ['aktienmarkt', 'amp', 'eigentlich', 'einsteigen', 'erinnern']...)


In [77]:
def create_document_matrix(tokens, id2word):
  corpus = []
  for text in tokens:
    corpus.append(id2word.doc2bow(text))
  return corpus
#passing the dataframe column having tokens and dictionary
de_corpus = create_document_matrix(de_df.tokens, de_id2word)
# # print(new_df.tokens[0])
# print(corpus[0])

In [80]:
de_lda_model = gensim.models.ldamodel.LdaModel(corpus=de_corpus, id2word=de_id2word, num_topics=10, random_state=100)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [82]:
def get_lda_topics(model, num_topics, top_n_words):
     word_dict = {}
     for i in range(num_topics):
         word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in model.show_topic(i, topn = top_n_words)];
 
     return pd.DataFrame(word_dict)
get_lda_topics(de_lda_model,10,20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,mehr,ja,mal,heute,ja,wissenschaft,amp,dauerstellen,mehr,wisszeitvg
1,amp,immer,schon,mal,mal,mehr,wissenschaft,ja,wissenschaft,promotion
2,jahre,stellen,wissenschaft,gute,schon,arbeitsbedingungen,forschung,stellen,gibt,mehr
3,mal,jahren,geht,mehr,letzten,dauerstellen,geht,mehr,forschung,dauerstellen
4,hanna,gut,macht,danke,uni,menschen,ja,gibt,stellen,ja
5,gibt,mehr,wirklich,wäre,n,reform,endlich,wisszeitvg,amp,wissenschaft
6,dürfen,müssen,eigentlich,evaluation,wäre,gute,lehre,tenure,lehre,immer
7,nein,gerade,wer,frage,ichbinhanna,problem,hochschule,track,deutschland,befristet
8,arbeiten,u,frau,bildung,jahre,amp,uni,lösung,mittelbau,jahre
9,wisszeitvg,gibt,system,uhr,gibt,sehen,wann,rede,stelle,wäre


In [83]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, de_corpus, de_id2word, mds='mmds', R=30)
vis

ValidationError: ignored

### ENGLISH PRE-PROCESSING

In [39]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [41]:
def to_lowercase(text):
    return text.lower()

en_df.text = en_df.text.apply(to_lowercase)

en_df.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type,language
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,thread cruel #precarity german #academia. the ...,5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original,en
105,2021-06-12T21:15:23.000Z,1403823267115376649,1403823267115376649,1132055796571877376,thread summarizing video german federal minist...,1,0,9,1,"[{'type': 'quoted', 'id': '1403006909079293953'}]","['IchbinHanna', 'academia', 'Germany', 'toxica...",original,en
125,2021-06-11T22:32:52.000Z,1403480380632674307,1403480380632674307,1132055796571877376,on job #precarity german #academia impact #fam...,0,0,1,0,"[{'type': 'quoted', 'id': '1402963684691955714'}]","['precarity', 'academia', 'family', 'mentalhea...",original,en
215,2021-06-10T19:07:52.000Z,1403066401611472902,1403066401611472902,1132055796571877376,#academictwitter @academicchatter #acertaindeg...,0,0,1,0,"[{'type': 'quoted', 'id': '1403059508021284864'}]","['AcademicTwitter', 'acertaindegreeofflexibili...",original,en
238,2021-06-10T15:04:44.000Z,1403005214974640143,1403005214974640143,1132055796571877376,#ichbinhanna: german researchers without perma...,7,0,15,0,"[{'type': 'quoted', 'id': '1402994337009442820'}]","['IchbinHanna', 'academia', 'AcademicTwitter',...",original,en


In [43]:
def remove_mentions_and_tags(text): #Removing mentions and hashtags
    text = re.sub(r'@\S*', '', text)
    return re.sub(r'#\S*', '', text)

en_df.text = en_df.text.apply(remove_mentions_and_tags)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [42]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [44]:
en_df['text'] = en_df['text'].str.replace(r'[^\w\s]+', '')

<ipython-input-44-87f5f943dbde>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  en_df['text'] = en_df['text'].str.replace(r'[^\w\s]+', '')
<ipython-input-44-87f5f943dbde>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_df['text'] = en_df['text'].str.replace(r'[^\w\s]+', '')


In [ ]:
en_df['text'] = en_df['text'].str.replace('\d+', '')

In [45]:
en_df

,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type,language
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,thread cruel german the debate started 8 day...,5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original,en
105,2021-06-12T21:15:23.000Z,1403823267115376649,1403823267115376649,1132055796571877376,thread summarizing video german federal minist...,1,0,9,1,"[{'type': 'quoted', 'id': '1403006909079293953'}]","['IchbinHanna', 'academia', 'Germany', 'toxica...",original,en
125,2021-06-11T22:32:52.000Z,1403480380632674307,1403480380632674307,1132055796571877376,on job german impact life httpstcosfzr...,0,0,1,0,"[{'type': 'quoted', 'id': '1402963684691955714'}]","['precarity', 'academia', 'family', 'mentalhea...",original,en
215,2021-06-10T19:07:52.000Z,1403066401611472902,1403066401611472902,1132055796571877376,httpstcoxtqih3qy0v,0,0,1,0,"[{'type': 'quoted', 'id': '1403059508021284864'}]","['AcademicTwitter', 'acertaindegreeofflexibili...",original,en
238,2021-06-10T15:04:44.000Z,1403005214974640143,1403005214974640143,1132055796571877376,german researchers without permanent post co...,7,0,15,0,"[{'type': 'quoted', 'id': '1402994337009442820'}]","['IchbinHanna', 'academia', 'AcademicTwitter',...",original,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
308092,2022-10-04T14:01:01.000Z,1577297743663583232,1577176553666273280,2474312910,,0,0,0,0,"[{'type': 'replied_to', 'id': '157717655366627...",['IchbinHanna'],original,en
308243,2022-09-13T08:48:00.000Z,1569608824117465088,1569608824117465088,1554860518900486144,the together host seminar forever postdoc n...,3,0,9,0,[],['WissZeitGesetz'],original,en
308292,2021-09-17T14:38:18.000Z,1438874960110424072,1438766234489794560,32800037,theres studies made worse be pumping phd stu...,0,1,0,0,"[{'type': 'replied_to', 'id': '143887451160512...",[],original,en
308293,2021-09-17T14:36:31.000Z,1438874511605125123,1438766234489794560,743696568939753475,i still cant get titled youngins juniorprofe...,0,1,0,0,"[{'type': 'replied_to', 'id': '143887433770928...",[],original,en


In [46]:
english_stop_words = stopwords.words('english')

def stop_word_en_removal(x):
    token = x.split()
    return ' '.join([w for w in token if not w in english_stop_words])

en_df.text = en_df.text.apply(stop_word_en_removal)
en_df.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type,language
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,thread cruel german debate started 8 days ago ...,5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original,en
105,2021-06-12T21:15:23.000Z,1403823267115376649,1403823267115376649,1132055796571877376,thread summarizing video german federal minist...,1,0,9,1,"[{'type': 'quoted', 'id': '1403006909079293953'}]","['IchbinHanna', 'academia', 'Germany', 'toxica...",original,en
125,2021-06-11T22:32:52.000Z,1403480380632674307,1403480380632674307,1132055796571877376,job german impact life httpstcosfzr6i5fse,0,0,1,0,"[{'type': 'quoted', 'id': '1402963684691955714'}]","['precarity', 'academia', 'family', 'mentalhea...",original,en
215,2021-06-10T19:07:52.000Z,1403066401611472902,1403066401611472902,1132055796571877376,httpstcoxtqih3qy0v,0,0,1,0,"[{'type': 'quoted', 'id': '1403059508021284864'}]","['AcademicTwitter', 'acertaindegreeofflexibili...",original,en
238,2021-06-10T15:04:44.000Z,1403005214974640143,1403005214974640143,1132055796571877376,german researchers without permanent post comm...,7,0,15,0,"[{'type': 'quoted', 'id': '1402994337009442820'}]","['IchbinHanna', 'academia', 'AcademicTwitter',...",original,en
